In [1]:
import torch
print(torch.__version__)

2.1.0


In [2]:
# import cv2
# import numpy as np
# from mtcnn import MTCNN
from deep_sort_realtime.deepsort_tracker import DeepSort
# import face_recognition
# import os
# import numpy as np
# import joblib

import cv2
import numpy as np
import face_recognition
import os
import numpy as np
import joblib

In [3]:
# Initialize DeepSORT tracker
tracker = DeepSort(max_age=10)

In [4]:
# Load pre-trained Haarcascades model for face detection in OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [5]:
# Open video file
video_path = "Class Room Entrance.mp4"
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [6]:
# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use "mp4v" for mp4 format
output_path = "output_video.mp4"
out = cv2.VideoWriter(output_path, fourcc, 20.0, (frame_width, frame_height))

In [7]:
frame_id = 0
skip_frames = 2  # Skip the frames

In [8]:
# Size threshold for faces (adjust as needed)
min_face_size = 10000  # For example, minimum area of 10000 pixels

In [9]:
# Dictionary to track whether the face image and embedding have been saved for each ID
saved_data = {}

In [10]:
# Create a directory to save face images
os.makedirs("faces(Vnew4)", exist_ok=True)

In [11]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1

    # Skip frames until reaching the nth frame
    if frame_id <= skip_frames:
        continue

    # Convert the frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Update tracker with the detected faces
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Convert faces to the list of tuples format (x, y, x + w, y + h)
    bbs = [((x, y, w, h), 1.0) for (x, y, w, h) in faces]
    print(bbs)
    # Update tracker with the detected faces
    tracks = tracker.update_tracks(bbs, frame=frame)

    # Draw bounding boxes and IDs on the frame
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id

        # Check if data for this ID has already been saved
        if track_id in saved_data:
            continue

        ltrb = track.to_ltrb()

        # Check face size
        face_area = (ltrb[2] - ltrb[0]) * (ltrb[3] - ltrb[1])
        if face_area < min_face_size:
            continue  # Skip small faces

        # Draw bounding box
        cv2.rectangle(frame, (int(ltrb[0]), int(ltrb[1])), (int(ltrb[2]), int(ltrb[3])), (0, 255, 0), 2)

        # Get facial landmarks using face_recognition library
        face_locations = [(int(ltrb[1]), int(ltrb[2]), int(ltrb[3]), int(ltrb[0]))]  # (top, right, bottom, left)
        landmarks = face_recognition.face_landmarks(frame, face_locations)

        if landmarks:
            # Get face embedding
            face_encoding = face_recognition.face_encodings(frame, face_locations)[0]

            # Check if the face embedding is not empty before saving
            if len(face_encoding) > 0:
                # Compare with existing embeddings
                match = False
                for saved_id, saved_info in saved_data.items():
                    saved_embedding = saved_info["embedding"]
                    # Compare the face embeddings using face_recognition library
                    results = face_recognition.compare_faces([saved_embedding], face_encoding)

                    # Check if there is a match
                    if any(results):
                        match = True
                        break

                # If no match, save the face embedding
                if not match:
                    # Save the face embedding
                    saved_data[track_id] = {
                        "embedding": face_encoding,
                        "image_path": f"faces(Vnew4)/{track_id}_frame{frame_id}.png"
                    }

                    # Get the face bounding box coordinates
                    top, right, bottom, left = int(ltrb[1]), int(ltrb[2]), int(ltrb[3]), int(ltrb[0])

                    # Check if the bounding box coordinates are within the frame boundaries
                    if 0 <= top < frame.shape[0] and 0 <= left < frame.shape[1] and top < bottom and left < right:
                        # Capture the face image
                        face_image = frame[top:bottom, left:right]

                        # Check if the face image is not empty before saving
                        if not face_image.size == 0:
                            # Save the face image
                            cv2.imwrite(saved_data[track_id]["image_path"], face_image)

                            # Draw label
                            label = f"ID: {track_id}"
                            cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Write the frame with bounding boxes to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

[((578, 66, 94, 94), 1.0), ((339, 50, 39, 39), 1.0), ((206, 54, 49, 49), 1.0), ((475, 59, 39, 39), 1.0)]
[((566, 59, 104, 104), 1.0), ((646, 323, 65, 65), 1.0), ((336, 49, 40, 40), 1.0), ((472, 58, 38, 38), 1.0), ((201, 62, 48, 48), 1.0)]
[((646, 323, 65, 65), 1.0), ((566, 59, 104, 104), 1.0), ((337, 49, 39, 39), 1.0), ((472, 58, 38, 38), 1.0), ((201, 62, 49, 49), 1.0)]
[((564, 57, 105, 105), 1.0), ((333, 50, 38, 38), 1.0), ((468, 58, 36, 36), 1.0), ((195, 65, 51, 51), 1.0)]
[((194, 69, 47, 47), 1.0), ((573, 58, 105, 105), 1.0), ((328, 50, 39, 39), 1.0), ((463, 57, 37, 37), 1.0)]
[((191, 68, 49, 49), 1.0), ((568, 64, 107, 107), 1.0), ((325, 51, 40, 40), 1.0), ((460, 57, 37, 37), 1.0)]
[((187, 65, 53, 53), 1.0), ((582, 70, 98, 98), 1.0), ((456, 58, 37, 37), 1.0)]
[((187, 64, 54, 54), 1.0), ((582, 71, 97, 97), 1.0), ((456, 58, 37, 37), 1.0)]
[((188, 69, 48, 48), 1.0), ((566, 72, 113, 113), 1.0), ((452, 58, 38, 38), 1.0), ((579, 54, 45, 45), 1.0)]
[((185, 72, 53, 53), 1.0), ((577, 80, 96,

In [12]:
saved_data

{'1': {'embedding': array([-0.13421801,  0.06550343,  0.10200148, -0.06876689, -0.02253696,
         -0.1085065 ,  0.05669782, -0.13979103,  0.18694441, -0.12699856,
          0.19196422, -0.02159148, -0.19425975, -0.10084616,  0.01426439,
          0.1041853 , -0.21440284, -0.17574665,  0.01153956, -0.09237312,
          0.06596953, -0.01437487, -0.00376997,  0.10222512, -0.15175369,
         -0.28846145, -0.10241228, -0.10542828, -0.01758523,  0.04988102,
          0.04385513,  0.11574474, -0.23029071,  0.0361933 , -0.05725952,
          0.07700047,  0.05502328, -0.02080932,  0.12716512,  0.00275872,
         -0.21959472, -0.0918505 ,  0.01643496,  0.20845069,  0.184983  ,
         -0.03650468,  0.01235081,  0.02144817,  0.08635779, -0.20907633,
          0.05100206,  0.14814508,  0.06494486,  0.04058941, -0.03995156,
         -0.21103714, -0.05373716,  0.0480428 , -0.19399135,  0.03659448,
          0.0132055 , -0.21311525, -0.09675668, -0.06167463,  0.1853414 ,
          0.12843439

In [13]:
# Load the age prediction model
loaded_age_model = joblib.load('age_model.pkl')

# Load the gender classification model
loaded_gender_model = joblib.load('gender_model.pkl')

# Lists to store predicted ages and genders
age_array = []
gender_array = []

# Iterate through saved_data for age and gender prediction
for track_id, info in saved_data.items():
    # Get the face embedding
    face_embedding = info["embedding"]

    # Reshape the face embedding to match the input shape expected by the models
    face_embedding_reshaped = face_embedding.reshape(1, -1)
    print(face_embedding_reshaped)
    # Predict age
    predicted_age = loaded_age_model.predict(face_embedding_reshaped)[0]
    age_array.append(predicted_age)

    # Predict gender
    predicted_gender = loaded_gender_model.predict(face_embedding_reshaped)[0]
    gender_array.append(predicted_gender)

# Convert age_array and gender_array to numpy arrays
age_array = np.array(age_array)
gender_array = np.array(gender_array)

# Print or use the predicted age and gender arrays as needed
print("Predicted Ages:", age_array)
print("Predicted Genders:", gender_array)


[[-0.13421801  0.06550343  0.10200148 -0.06876689 -0.02253696 -0.1085065
   0.05669782 -0.13979103  0.18694441 -0.12699856  0.19196422 -0.02159148
  -0.19425975 -0.10084616  0.01426439  0.1041853  -0.21440284 -0.17574665
   0.01153956 -0.09237312  0.06596953 -0.01437487 -0.00376997  0.10222512
  -0.15175369 -0.28846145 -0.10241228 -0.10542828 -0.01758523  0.04988102
   0.04385513  0.11574474 -0.23029071  0.0361933  -0.05725952  0.07700047
   0.05502328 -0.02080932  0.12716512  0.00275872 -0.21959472 -0.0918505
   0.01643496  0.20845069  0.184983   -0.03650468  0.01235081  0.02144817
   0.08635779 -0.20907633  0.05100206  0.14814508  0.06494486  0.04058941
  -0.03995156 -0.21103714 -0.05373716  0.0480428  -0.19399135  0.03659448
   0.0132055  -0.21311525 -0.09675668 -0.06167463  0.1853414   0.12843439
  -0.11002966 -0.16685791  0.2770656  -0.20455904 -0.02360645  0.09857252
  -0.13774176 -0.14444269 -0.26136312  0.04626876  0.36467478  0.10507961
  -0.09125526  0.09248258 -0.21235135  0